In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the API key
api_key = os.getenv("API_KEY")

In [2]:
import requests

# Function to fetch weather data
def get_weather(city, api_key):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error:", response.status_code)
        return None

# Function to display weather data
def display_weather(data):
    if data:
        city = data["name"]
        temp = data["main"]["temp"]
        weather = data["weather"][0]["description"]
        humidity = data["main"]["humidity"]
        wind_speed = data["wind"]["speed"]

        print(f"Weather in {city}:")
        print(f"Temperature: {temp}°C")
        print(f"Condition: {weather}")
        print(f"Humidity: {humidity}%")
        print(f"Wind Speed: {wind_speed} m/s")
    else:
        print("No data to display.")

# # Main program
# if __name__ == "__main__":
#     city_name = input("Enter the city name: ")
#     api_key = "your_api_key_here"  # Replace with your OpenWeatherMap API key
#     weather_data = get_weather(city_name, api_key)
#     display_weather(weather_data)


In [4]:
city_name = 'Surabaya'

In [5]:
weather_data = get_weather(city_name, api_key)
display_weather(weather_data)

Weather in Surabaya:
Temperature: 26.33°C
Condition: moderate rain
Humidity: 88%
Wind Speed: 1.59 m/s
